In [1]:
import pandas as pd

# Loading data

In [36]:
page_view = pd.read_csv("data/lightning sample/lightning pageview sample/2022-06-05_LightningPageView.csv")

C:\Users\gabim\AppData\Local\Temp\ipykernel_39376\1656048912.py:1: DtypeWarning: Columns (5,11,13,17,18,19,20,22,24,28,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  page_view = pd.read_csv("data/lightning sample/lightning pageview sample/2022-06-05_LightningPageView.csv")


In [41]:
page_view = page_view[page_view['PAGE_ENTITY_ID'].notna()]

Many PAGE_ENTITY_ID values were NaN, so those values were filtered.

# Counting component occurrences

In [42]:
count = page_view['PAGE_CONTEXT'].value_counts(sort=True)
print(count.to_string())

one:recordHomeFlexipage2Wrapper                 378425
one:recordHomeFlexipage                         118307
one:recordActionWrapper                           7810
unknown                                           2206
runtime_platform_actions:quickActionWrapper        927
forceKnowledge:articleRedirect                     367
sfa:hierarchyFullView                               27
reports:reportBuilder                               23
forceChatter:feedElementFullView                     4
runtime_service_fieldservice:objectHierarchy         3
force:recordLandingPage                              2


For each component, this is the number of times it appeared on a PageView event.

# Associating Page Entity ID and Page Context

From the PageView docs:
- PAGE_ENTITY_ID: The unique entity identifier of the event.
- PAGE_CONTEXT: The name of the component hosting the main content of the page.

This analysis will use these fields to understand which components are used in more than one page.

## Filtering table columns

In [55]:
resume_page_view = page_view[['PAGE_ENTITY_ID','PAGE_CONTEXT']]
resume_page_view

,PAGE_ENTITY_ID,PAGE_CONTEXT
2,5006P000005AtMgQAK,one:recordHomeFlexipage2Wrapper
3,0WO6P000001kNGgWAM,one:recordHomeFlexipage
4,5006P000005UJXVQA4,one:recordHomeFlexipage2Wrapper
5,5006P000005AsJ2QAK,one:recordHomeFlexipage2Wrapper
7,5000b00001Q2AEaAAN,one:recordHomeFlexipage2Wrapper
...,...,...
751604,ka02R0000008H4zQAE,one:recordHomeFlexipage
751605,0010b00001u0lKHAAY,one:recordHomeFlexipage2Wrapper
751607,02i2R00000Kh83JQAR,one:recordHomeFlexipage2Wrapper
751608,5006P000005EUjBQAW,one:recordHomeFlexipage2Wrapper


The data frame now has only the columns we are interested for our analysis.

## Grouping PAGE_ENTITY_ID

For each Page Context, we need to group the Page Entity IDs associated with it.

In [56]:
group_resume_pv = resume_page_view.groupby(['PAGE_CONTEXT']).agg({'PAGE_ENTITY_ID': 'unique'})
group_resume_pv.head(20)

,PAGE_ENTITY_ID
PAGE_CONTEXT,
force:recordLandingPage,"[0696P000003v7whQAA, 0696P000002z4lRQAQ]"
forceChatter:feedElementFullView,"[0D56P00000YcGEzSAN, 0D56P00000ZdWTYSA3, 0D56P..."
forceKnowledge:articleRedirect,"[kA02R0000005cMdSAI, kA02R0000005JPZSA2, kA02R..."
one:recordActionWrapper,"[5006P000005Xce7QAC, 5006P000005XerDQAS, 00T6P..."
one:recordHomeFlexipage,"[0WO6P000001kNGgWAM, 0WO6P0000017wuoWAA, 00T6P..."
one:recordHomeFlexipage2Wrapper,"[5006P000005AtMgQAK, 5006P000005UJXVQA4, 5006P..."
reports:reportBuilder,"[00O6P000000ZNSpUAO, 00O0b000004kTazEAE, 00O6P..."
runtime_platform_actions:quickActionWrapper,"[a0L6P000000spv6UAA, 0012R00002AKWthQAH, a0L6P..."
runtime_service_fieldservice:objectHierarchy,"[02i2R00000N4IUfQAN, 02i1P000007meR2QAI, 02i2R..."


Now every component has a list of Page Entity IDs that are associated with it.

## Unique Page Entity IDs

Some Page Entity IDs occur more than once, and we only need to know if the component is present in that page, not how many times it was used. To do that we will create a set (a data type that does not allow repetitions).

In [67]:
set_of_unique_page_entity_ids = list(page_view['PAGE_ENTITY_ID'].unique())

In [68]:
def countingPageViews(list_of_page_entity_ids): 
    count = 0
    for i in list_of_page_entity_ids:
        count += i in set_of_unique_page_entity_ids
    return count

We can apply the *countingPageViews* function to our data frame and every component will have a list of unique Page Entity IDs that use the component.

In [69]:
group_resume_pv['pageEntityIDCounts'] = group_resume_pv['PAGE_ENTITY_ID'].apply(countingPageViews)
group_resume_pv.head(20)

,PAGE_ENTITY_ID,pageEntityIDCounts
PAGE_CONTEXT,,
force:recordLandingPage,"[0696P000003v7whQAA, 0696P000002z4lRQAQ]",2
forceChatter:feedElementFullView,"[0D56P00000YcGEzSAN, 0D56P00000ZdWTYSA3, 0D56P...",4
forceKnowledge:articleRedirect,"[kA02R0000005cMdSAI, kA02R0000005JPZSA2, kA02R...",119
one:recordActionWrapper,"[5006P000005Xce7QAC, 5006P000005XerDQAS, 00T6P...",5918
one:recordHomeFlexipage,"[0WO6P000001kNGgWAM, 0WO6P0000017wuoWAA, 00T6P...",45487
one:recordHomeFlexipage2Wrapper,"[5006P000005AtMgQAK, 5006P000005UJXVQA4, 5006P...",90644
reports:reportBuilder,"[00O6P000000ZNSpUAO, 00O0b000004kTazEAE, 00O6P...",12
runtime_platform_actions:quickActionWrapper,"[a0L6P000000spv6UAA, 0012R00002AKWthQAH, a0L6P...",852
runtime_service_fieldservice:objectHierarchy,"[02i2R00000N4IUfQAN, 02i1P000007meR2QAI, 02i2R...",3


## Top 10 most used components

Listed below the 10 components with more Page Entity IDs associated.

In [71]:
group_resume_pv.sort_values(by='pageEntityIDCounts', ascending=False).head(10)

,PAGE_ENTITY_ID,pageEntityIDCounts
PAGE_CONTEXT,,
one:recordHomeFlexipage2Wrapper,"[5006P000005AtMgQAK, 5006P000005UJXVQA4, 5006P...",90644
one:recordHomeFlexipage,"[0WO6P000001kNGgWAM, 0WO6P0000017wuoWAA, 00T6P...",45487
one:recordActionWrapper,"[5006P000005Xce7QAC, 5006P000005XerDQAS, 00T6P...",5918
runtime_platform_actions:quickActionWrapper,"[a0L6P000000spv6UAA, 0012R00002AKWthQAH, a0L6P...",852
forceKnowledge:articleRedirect,"[kA02R0000005cMdSAI, kA02R0000005JPZSA2, kA02R...",119
sfa:hierarchyFullView,"[0010b000029iT41AAE, 0036P000004KOB5QAO, 0032R...",14
reports:reportBuilder,"[00O6P000000ZNSpUAO, 00O0b000004kTazEAE, 00O6P...",12
forceChatter:feedElementFullView,"[0D56P00000YcGEzSAN, 0D56P00000ZdWTYSA3, 0D56P...",4
runtime_service_fieldservice:objectHierarchy,"[02i2R00000N4IUfQAN, 02i1P000007meR2QAI, 02i2R...",3
